# Polecenie
Celem zadania drugiego jest zaimplementowanie metody eliminacji Jordana rozwiązywania układu $N$ równań liniowych z $N$ niewiadomymi.

- [X] Maksymalna ilość równań może być ograniczona z góry,
- [X] ale skonstruowany algorytm ma być uniwersalny,
- [X] a użytkownik ma mieć możliwość wyboru ilości równań w trakcie pracy programu.
- [X] Należy zapewnić możliwość wygodnego wprowadzania współczynników układu równań poprzez wczytywanie ich z pliku.

- [X] Program musi automatycznie wybierać element podstawowy i wykrywać sytuacje, kiedy układ jest sprzeczny albo nieoznaczony.
- [X] W sprawozdaniu należy zamieścić informację jak zachowuje się program w przypadku układów sprzecznych i nieoznaczonych.

# Program
Zdecydowaliśmy się nie ograniczać maksymalnej ilości równań.

W naszym programie wykorzystujemy biblioteki `csv` i `numpy`.
Poza tym w komórce poniżej widać 

In [ ]:
# IMPORTS & CONSTANTS
import csv
import numpy as np; from numpy import array
import numpy.testing as npt

example_path = "ex/"
UNSOLVABLE_MSG = "Układ nie jest rozwiązywalny"
WRONG_MSG = "Układ jest sprzeczny"
DEBUG = True

Do programu równania można ładować plikiem CSV, do czego służy funkcja `read_csv`:

In [ ]:
def read_csv(file_path): 
    fat = np.genfromtxt(file_path, delimiter=',')
    b = array([ row[-1] for row in fat ])
    A = np.delete(fat, -1, axis=1)
    if A.shape[0] != A.shape[1]: # is not A 1x1?
        raise IndexError("Podany układ równań nie jest odpowiedni")
    return A, b
if DEBUG: read_csv("ex/a.csv")

Sercem naszego programu jest funkcja `eliminate`.

Rozwiązuje ona układ równań zwracając wektor $x$.
Ponadto rozpoznaje układy sprzeczne i nierozwiązywalne i wtedy podnosi błąd, co widać w pierwszym `if`.

In [ ]:
def eliminate(data):
    A = data[0]; b = data[1]; n = len(A)

    # Check for exceptions
    if np.linalg.det(A) == 0: # Is this thing solvable?
        if np.all(np.dot(A, np.linalg.lstsq(A, b, rcond=None)[0]) != b): # rcond to silence the error
            raise ValueError(WRONG_MSG) # it's unsolvable because it's wrong (special case)
        else: raise ValueError(UNSOLVABLE_MSG) # general unsolvable case
    
    # Augmenting the matrix and converting to float64
    augmented_matrix = np.column_stack((A.astype(float), b.astype(float)))

    # Forward Elimination with Partial Pivoting
    for pivot_row in range(n):
        # Partial pivoting: find the row with the largest absolute value of pivot element
        max_row = pivot_row
        for row in range(pivot_row + 1, n):
            if abs(augmented_matrix[row][pivot_row]) > abs(augmented_matrix[max_row][pivot_row]):
                max_row = row
        augmented_matrix[[pivot_row, max_row]] = augmented_matrix[[max_row, pivot_row]] # Swap the rows
        if augmented_matrix[pivot_row][pivot_row] == 0: # Check if pivot element is zero
            raise ValueError("Natrafiono na zero pivot podczas częściwego pivotowania")
        
        for row in range(pivot_row + 1, n):
            factor = augmented_matrix[row][pivot_row] / augmented_matrix[pivot_row][pivot_row]
            augmented_matrix[row] -= factor * augmented_matrix[pivot_row]

    # Back Substitution
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        x[i] = augmented_matrix[i][n] / augmented_matrix[i][i]
        for j in range(i - 1, -1, -1): augmented_matrix[j][n] -= augmented_matrix[j][i] * x[i]
    return x

# Testy jednostkowe
Jako profesjonaliści piszemy testy jednostkowe.
## Testy z dobrymi danymi

In [ ]:
npt.assert_allclose(array([ 1, 2, 3 ]), eliminate(read_csv(example_path + "a.csv")),         err_msg="Nie działa (a)")
npt.assert_allclose(array([ 2, -3, 1.5, 0.5 ]), eliminate(read_csv(example_path + "d.csv")), err_msg="Nie działa (d)")
npt.assert_allclose(array([ 1, 3, -4, 5 ]), eliminate(read_csv(example_path + "f.csv")),     err_msg="Nie działa (f)")
npt.assert_allclose(array([ 1, 2, 3 ]), eliminate(read_csv(example_path + "h.csv")),         err_msg="Nie działa (h)")
npt.assert_allclose(array([ 1, 1, 1 ]), eliminate(read_csv(example_path + "j.csv")),         err_msg="Nie działa (j)")

### Przypadek specjalny -- zera

In [ ]:
npt.assert_allclose(array([ 7, 5, 3 ]), eliminate(read_csv(example_path + "g.csv")), err_msg="Nie działa z zerami (g)")

## Testy ze złymi danymi
Układy nierozwiązywalne w naszym programie są rozpoznawane jako układy sprzeczne bądź ogólnie nierozwiązywalne.

Te drugie w przykładach są reprezentowane przez układy nieoznaczone.

In [ ]:
## 3. Testy sprzecznych
try:
    eliminate(read_csv(example_path + "c.csv"))
    eliminate(read_csv(example_path + "e.csv"))
except ValueError as e:
    assert str(e) == WRONG_MSG

In [ ]:
## 4. Testy nieoznaczonych
try:
    eliminate(read_csv(example_path + "b.csv"))
    eliminate(read_csv(example_path + "i.csv"))
except ValueError as e:
    assert str(e) == WRONG_MSG or str(e) == UNSOLVABLE_MSG

## Przykładowy błąd
Specjalnie na potrzeby sprawozdania pokazujemy jeden z błędów:

In [ ]:
zly = (array([[1,1,1],
             [1,1,2],
             [1,1,3]]),
      array([1,1,1]))
eliminate(zly)